# Plot influenza mortality
Data taken from [Simonsen et al (1998)](https://academic.oup.com/jid/article/178/1/53/919896).

First, import Python modules:

In [1]:
import os

import altair as alt

import pandas as pd

Now create a data frame with the data from [Simonsen et al (1998)](https://academic.oup.com/jid/article/178/1/53/919896):

In [2]:
mortality = pd.DataFrame(
    [
        # from Table 1
        (1918, "H1N1", 529, 166, 546),
        (1919, "H1N1", 93, 175, 89),
        (1921, "H1N1", 16, 93, 12),
        (1922, "H1N1", 37, 280, 24),
        (1925, "H1N1", 26, 221, 15),
        (1928, "H1N1", 84, 594, 55),
        (1936, "H1N1", 18, 101, 12),
        (1943, "H1N1", 14, 123, 4.8),
        # only use data from prior to 1957 due to gap in evolution
        # (1977, "H1N1", 3.8, 30, 0.6),
        # (1983, "H1N1", 1.5, 16, 0),
        # (1986, "H1N1", 0.7, 5.7, 0),
        # (1988, "H1N1", 2.1, 15, 0.1),
        # from Table 3
        (1968, "H3N2", 8.1, 44, 4.3),
        (1969, "H3N2", 3.3, 16, 1.9),
        (1970, "H3N2", 0.7, 2.8, 0.4),
        (1971, "H3N2", 4.2, 29, 1.5),
        (1972, "H3N2", 3.8, 21, 1.4),
        (1974, "H3N2", 3.0, 19, 0.8),
        (1975, "H3N2", 5.5, 42, 0.8),
        (1977, "H3N2", 3.8, 30, 0.6),
        (1980, "H3N2", 5.2, 41, 0.6),
        (1982, "H3N2", 2.0, 19, 0),
        (1984, "H3N2", 3.4, 29, 0.1),
        (1987, "H3N2", 3.1, 24, 0.1),
        (1989, "H3N2", 4.1, 30, 0.3),
        (1991, "H3N2", 2.6, 19, 0.2),
        (1992, "H3N2", 2.4, 16, 0.2),
        (1993, "H3N2", 3.9, 29, 0.3),
        (1994, "H3N2", 2.0, 15, 0.1),
    ],
    columns=["year", "subtype", "all ages", "65+ years", "<65 years"],
).assign(
    year=lambda x: pd.to_datetime(x["year"], format="%Y"),
    subtype=lambda x: x["subtype"] + " influenza",
)

mortality

,year,subtype,all ages,65+ years,<65 years
0,1918-01-01,H1N1 influenza,529.0,166.0,546.0
1,1919-01-01,H1N1 influenza,93.0,175.0,89.0
2,1921-01-01,H1N1 influenza,16.0,93.0,12.0
3,1922-01-01,H1N1 influenza,37.0,280.0,24.0
4,1925-01-01,H1N1 influenza,26.0,221.0,15.0
5,1928-01-01,H1N1 influenza,84.0,594.0,55.0
6,1936-01-01,H1N1 influenza,18.0,101.0,12.0
7,1943-01-01,H1N1 influenza,14.0,123.0,4.8
8,1968-01-01,H3N2 influenza,8.1,44.0,4.3
9,1969-01-01,H3N2 influenza,3.3,16.0,1.9


Make tidy version of data:

In [3]:
mortality_tidy = mortality.melt(
    id_vars=["year", "subtype"],
    var_name="age group",
    value_name="mortality",
)

mortality_tidy

,year,subtype,age group,mortality
0,1918-01-01,H1N1 influenza,all ages,529.0
1,1919-01-01,H1N1 influenza,all ages,93.0
2,1921-01-01,H1N1 influenza,all ages,16.0
3,1922-01-01,H1N1 influenza,all ages,37.0
4,1925-01-01,H1N1 influenza,all ages,26.0
...,...,...,...,...
70,1989-01-01,H3N2 influenza,<65 years,0.3
71,1991-01-01,H3N2 influenza,<65 years,0.2
72,1992-01-01,H3N2 influenza,<65 years,0.2
73,1993-01-01,H3N2 influenza,<65 years,0.3


Plot data:

In [4]:
age_group_selection = alt.selection_multi(
    fields=["age group"],
    bind="legend",
    init=[{"age group": "all ages"}],
)

mortality_base = (
    alt.Chart(mortality_tidy)
    .encode(
        x=alt.X("year:T"),
        y=alt.Y("mortality", title="excess mortality per 100,000"),
        color=alt.Color(
            "age group",
            scale=alt.Scale(
                range=["#000000", "#E69F00", "#56B4E9"],
                domain=["all ages", "<65 years", "65+ years"],
            )
        ),
        tooltip=[c for c in mortality_tidy.columns],
    )
    .properties(width=500, height=140)
)

mortality_points = mortality_base.mark_point(filled=True, size=50, opacity=1)

mortality_lines = mortality_base.mark_line(size=1, strokeDash=[2, 2], opacity=1)

mortality_chart = (
    (mortality_points + mortality_lines)
    .facet(
        alt.Facet(
            "subtype",
            title=None,
            header=alt.Header(labelFontSize=13, labelPadding=0),
        ),
        title=alt.TitleParams(
            [
                "Influenza & pneumonia excess mortality in the United States, taken from Simonson et al, JID, (1998).",
                "The H1N1 pandemic occurred in 1918, and the H3N2 pandemic occurred in 1968.",
                "Click / shift-click on the legend to show data for specific age groups."
            ],
            orient="bottom",
            fontSize=12,
            fontWeight="normal",
            dy=8,
        ),
        columns=1,
        spacing=12,
    )
    .configure_axis(grid=False)
    .resolve_scale(y="independent", x="independent")
    .add_selection(age_group_selection)
    .transform_filter(age_group_selection)
)

mortality_chart.save("index.html")

mortality_chart

/fh/fast/bloom_j/software/miniconda3/envs/BloomLab/lib/python3.8/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.FacetChart(...)